Initially we will install all of our required packages after lots of testings


In [1]:
!pip install qdrant-client langchain tiktoken openai mistralai sentence-transformers langchain-community  llama-cpp-python  -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 13.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 34.0 MB/s eta 0:00:00
     ━━━━

Do the vectorization with Qdrant for our database

In [2]:
from langchain.vectorstores import  qdrant
import qdrant_client
import os

In [3]:
# create a qdrant client
os.environ['QDRANT_HOST'] = "https://62388b95-0196-4274-82ac-c80ca77a19eb.us-east4-0.gcp.cloud.qdrant.io:6333"
os.environ['QDRANT_API_KEY'] = "QSigZ7f8mC3vyCsTUqL3ri7CixTzg1-kMQ4IRU7sFYI6Ng3k0cW1dw"

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key= os.getenv("QDRANT_API_KEY")
)



In [4]:
#create collection
from qdrant_client import QdrantClient, models
os.environ['QDRANT_COLLECTION_NAME'] = "txt-stories-final-xiii"

vectors_config=models.VectorParams(
    size=384,
    distance=models.Distance.COSINE)
client.create_collection(
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    vectors_config=vectors_config,
)


True

In [5]:
# create your vector store
from langchain.vectorstores import Qdrant
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Qdrant(
        client=client,
        collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
        embeddings=embeddings
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the langchain-qdrant package and should be used instead. To use it run `pip install -U langchain-qdrant` and import as `from langchain_qdrant import Qdrant`.
  warn_deprecated(


In [7]:
# add documents to your vector database

from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

with open("/content/drive/MyDrive/stories.txt") as f:
    raw_text = f.read()

texts = get_chunks(raw_text)

vectorstore.add_texts(texts)

['84f08af4ce0447a3b0561988d9a723c9',
 'dd25c5c19d6b478abd11eedad921c948',
 'bb4ec488e701403cb677cba2a09100f6',
 'f2e7833fde2548948cd0b5284b7c6a3d',
 '0a217b60ef7a4910bd68e36f1e53d0c1',
 '5aade7f9d0d64af6812990c5d27f5d7b',
 'a8dfa98cc15c43d48ff5f713bf5d29ee',
 '0b86271b40464b7c87ebcfc85d9cb7e1',
 '064d3ab47fd649ce936f327025066d81',
 'e8dec83b372543369f849758ca4cd488',
 'a37bb793d39c4ed3b3647aa9b6e3fdd5',
 'c5e3bc1b1040464f964766dfbcc83b0f',
 '4538e0a408804d85b3df9cf2ede0e103',
 'c77bb0d8fef2447ca55e42066a10c61d',
 '7709fbefb4d8495c91f16310448c24c0',
 'aff301cd4b764f1d8837216366145bb9',
 '33be57879b884c55a5d42cdf6cf2d322',
 '858226dc5f6c4e268eb639662ea8616b',
 'd275b1c2852748128e4eeb4c037d22a6',
 '29121459d5094d41b789d9ed8badccfe',
 '2ba33dbf091a46c2b9c642303d12e073',
 '539025753223461783aa224bb650650b',
 '0591e238277847d4a748ee5766566392',
 '6187076c84fd480cbdcd810bb888e0d2',
 '51cc19445b8b4a5e89e371f05a6ff44a',
 'b7e6e179a8fa465989baab785b8716a5',
 '9e376cdc037f4cb19d19d27212a0559b',
 

In [8]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q5_K_M.gguf",
    temperature=0.25,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

In [9]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(search_kwargs={"k": 2}))

In [10]:
query = "Tell me about Dr. Amelia Hart from The Chronicles of the Cosmic Rift."
qa.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    6292.26 ms
llama_print_timings:      sample time =     109.72 ms /   150 runs   (    0.73 ms per token,  1367.13 tokens per second)
llama_print_timings: prompt eval time =  275702.17 ms /   438 tokens (  629.46 ms per token,     1.59 tokens per second)
llama_print_timings:        eval time =  129251.23 ms /   149 runs   (  867.46 ms per token,     1.15 tokens per second)
llama_print_timings:       total time =  405930.35 ms /   587 tokens


" Dr. Amelia Hart is an astrophysicist who was one of the leading minds working on Elysium, a project that aimed to create a stable wormhole for humanity's survival. Her research focused on the mysterious phenomenon known as the Cosmic Vortex, and she believed that these cosmic bridges held the key to humanity's future among the stars. Despite facing challenges and seemingly impossible odds, Dr. Hart was determined to unlock the secrets of the Cosmic Vortex and create a prototype device called the Elysium Portal. Her work and legacy served as a testament to human ingenuity and determination, inspiring generations to dream of the stars and strive for a better tomorrow."

In [11]:
query = "Tell me about Evelyn from The Eldorian Secrets"
qa.run(query)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6292.26 ms
llama_print_timings:      sample time =      43.93 ms /    60 runs   (    0.73 ms per token,  1365.68 tokens per second)
llama_print_timings: prompt eval time =  241998.38 ms /   394 tokens (  614.21 ms per token,     1.63 tokens per second)
llama_print_timings:        eval time =   53736.02 ms /    59 runs   (  910.78 ms per token,     1.10 tokens per second)
llama_print_timings:       total time =  296206.11 ms /   453 tokens


' Evelyn is a character in "The Chronicles of the Lost Kingdom" and is described as being kind-hearted and compassionate. She is also said to be intelligent and resourceful, which helps her in her quest to uncover the secrets of the lost kingdom of Eldoria.'